In [2]:
from matplotlib import cm, pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cross_val_score


def load_dataset(partition):
    X,y = fetch_openml('mnist_784', version=1, return_X_y=True)
    y = y.astype(int)[:int(len(X)*partition//1)]
    X = X/255
    X=X[:int(len(X)*partition//1)]
    return X,y

def tune_parameter(parname, parvalues ,cf ,k_fold,X,y):
    acc=[]
    for i in range(len(parvalues)):
        cf.set_params(**{parname:parvalues[i]})
        acc.append(cross_val_score(cf, X, y, cv=k_fold,scoring="accuracy"))
    return acc


In [3]:


X,y=load_dataset(.5)


In [4]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2)

In [ ]:
from numpy import arange

cl=RandomForestClassifier(n_estimators=100)
t_values=[.001,.0005,.00025,.0001,.00005]
print(t_values)
t_acc=tune_parameter("min_impurity_decrease",t_values,cl,10,X_train,y_train)

[0.001, 0.0005, 0.00025, 0.0001, 5e-05]


In [ ]:
from numpy import mean


print(mean(t_acc,axis=1))
t_accp=mean(t_acc,axis=1)

In [ ]:

fig, ax = plt.subplots()
x_place=arange(len(t_accp))
ax.plot(t_accp)

plt.xticks(x_place, t_values)



In [ ]:
cl=RandomForestClassifier(n_estimators=100,min_impurity_decrease=0.0001)
cl.fit(X_train,y_train)

clBase=RandomForestClassifier(n_estimators=100)
clBase.fit(X_train,y_train)


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,cl.predict(X_test)))
print(accuracy_score(y_test,clBase.predict(X_test)))

In [ ]:

fi=cl.feature_importances_
data=(fi/max(fi)).reshape(28,28)
fig, ax = plt.subplots()
l=ax.imshow(data, cmap=cm.jet, interpolation='nearest')
ax.set_title("Feature importance")
fig.colorbar(l,ax=ax)


In [ ]:
fi=clBase.feature_importances_
data=(fi/max(fi)).reshape(28,28)
fig, ax = plt.subplots()
l=ax.imshow(data, cmap=cm.jet, interpolation='nearest')
ax.set_title("Feature importance")
fig.colorbar(l,ax=ax)